In [1]:
import os
import sys
from scgt import GeoTiff
sys.path.append("..")

from ecoscape_layers.layers import RedList, LayerGenerator, reproject_shapefile
from ecoscape_layers.constants import EBIRD_INDIV_RANGE_PATH, EBIRD_INDIV_RANGE_LAYER
from config import REDLIST_KEY, EBIRD_KEY

In [2]:
species_list_path = "./inputs/test_species_list.txt"
terrain_path = "./inputs/test_terrain.tif"
terrain_codes_path = "./inputs/terrain_codes.csv"
species_range_folder = "./inputs/ebird_ranges"
output_folder = "./outputs"

crs = 'PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
resolution = 300
resampling = "near"
bounds = (-2214130.5469038677401841, 2459449.2663429058156908, -2174930.5469038677401841, 2504349.2663429058156908)
padding = 10000

refine_method = "forest"

In [3]:
# Define eBird-specific range map path and gpkg layer.
indiv_range_path = os.path.join(species_range_folder, EBIRD_INDIV_RANGE_PATH)
indiv_range_layer = EBIRD_INDIV_RANGE_LAYER

terrain_path = os.path.abspath(terrain_path)
terrain_codes_path = os.path.abspath(terrain_codes_path)

In [4]:
# Get the list of bird species from species_list_path.
with open(species_list_path) as file:
    species_list = file.read().splitlines()

# Generate output folder.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Generate species output folders.
for species in species_list:
    species_output_folder = os.path.join(output_folder, species)
    if not os.path.exists(species_output_folder):
        os.makedirs(species_output_folder)

In [5]:
redlist = RedList(REDLIST_KEY, EBIRD_KEY)
layer_generator = LayerGenerator(terrain_path, terrain_codes_path, crs, resolution, resampling,
                                     bounds, padding)

In [6]:
# Generate terrain layer.
layer_generator.generate_terrain()

In [7]:
# Obtain species habitat information from the IUCN Red List.
species_data = []

for species in species_list:
    sci_name = redlist.get_scientific_name(species)
    habs = redlist.get_habitats(sci_name)
    
    if len(habs) == 0:
        print("Skipping", species, "due to not finding info on IUCN Red List (perhaps a name mismatch with eBird)?")
        continue
    else:
        species_data.append({
            "name": species,
            "sci_name": sci_name,
            "habitats": habs
        })

In [8]:
# Download species ranges as shapefiles from eBird.
layer_generator.get_ranges_from_ebird(species_list_path, species_range_folder)

In [9]:
# Create the resistance table for each species.
all_map_codes = layer_generator.get_map_codes()
for species in species_data:
    code = species["name"]
    resistance_output_path = os.path.join(output_folder, code, f"{code}_resistance.csv")
    layer_generator.generate_resistance_table(species["habitats"], all_map_codes, resistance_output_path)

In [10]:
# Perform the intersection between the range and habitable terrain.
with GeoTiff.from_file(layer_generator.terrain_path) as ter:
    resolution = int(ter.dataset.transform[0])
    
    for species in species_data:
        if species == "":
            break

        code = species["name"]
        habitats = species["habitats"]

        if not os.path.isfile(indiv_range_path.format(code=code)):
            print("Skipping {code}, no associated range map found".format(code=code))
            continue

        range_shapes = reproject_shapefile(
            shapes_path=indiv_range_path.format(code=code),
            dest_crs=crs,
            shapes_layer=indiv_range_layer
        )

        if len(range_shapes) == 1:
            # Not a seasonal bird
            path = os.path.join(output_folder, code, f"habitat_2020_{resolution}_{resampling}_{refine_method}.tif")
            layer_generator.refine_habitat(ter, habitats=habitats, shapes=range_shapes[0], output_path=path, refine_method=refine_method)
        else:
            # Seasonal bird, different output for each shape
            for s in range_shapes:
                season = str(s["properties"]["season"])
                path = os.path.join(output_folder, code, f"{season}_habitat_2020_{resolution}_{resampling}_{refine_method}.tif")
                layer_generator.refine_habitat(ter, habitats=habitats, shapes=s, output_path=path, refine_method=refine_method)